In [1]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction

### Add Directories/ Load Files

In [10]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\ENTRAINMENT_PROJECT


In [25]:
fname = 'sub035_ses-2022011308165394_run-BrainSense20220113084500.mat'

raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        project_path,
        'data',
        'raw_data',
        'test',
        fname
    ),
    info = None
)

Creating RawArray with float64 data, n_channels=6, n_times=227375
    Range : 0 ... 227374 =      0.000 ...   909.496 secs
Ready.


c:\Users\mathiopv\Anaconda3\lib\site-packages\pymatreader\utils.py:230: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_4144\3320525455.py:3: RuntimeWarning: Importing FieldTrip data without an info dict from the original file. Channel locations, orientations and types will be incorrect. The imported data cannot be used for source analysis, channel interpolation etc.
  raw = mne.io.read_raw_fieldtrip(
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_4144\3320525455.py:3: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  raw = mne.io.read_raw_fieldtrip(
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_4144\3320525455.py:3: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_

In [26]:
#Plot TF plot with artefact rejected data
reload(dat_preproc)
%matplotlib qt
matplotlib.rc('font', size=10)
x = raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

In [30]:
fft_name = 'Sub035_3mfu_FFt'

In [28]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name),dpi = 300)

np.save(str(fft_file)+str(fft_name)+'.npy',Sxx)

In [29]:
d = {'onset': [370, 542, 816],
    'duration': [30, 30, 10],
    'description': ['StimOff','StimOn','Rebound']
}

epoch_df = pd.DataFrame(data = d)
print(epoch_df)

   onset  duration description
0    370        30     StimOff
1    542        30      StimOn
2    816        10     Rebound


In [33]:
reload(dat_preproc)
filt_dat = raw.get_data()
time_onsets = epoch_df
window = 250
noverlap = 0.5
ylim2 = 0.2

fig = plt.figure(figsize = (12,5))

plt.subplot(1,2,1)
title1 = 'Sub035_3mfu_LSTN_Contra'
ps1 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 0, 0.1, title1)

plt.subplot(1,2,2)
title2 = 'Sub035_3mfu_RSTN_Ipsi'
ps2 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 1, 0.5, title2)

plt.show()

In [34]:
ps_all = np.concatenate((ps1,ps2), axis = 0)
ps_df = pd.DataFrame(np.transpose(ps_all),
    columns = ['StimOff_LSTN','StimOn_LSTN','Rebound_LSTN',
    'StimOff_RSTN','StimOn_RSTN','Rebound_RSTN'],
    )
print(ps_df)

     StimOff_LSTN  StimOn_LSTN  Rebound_LSTN  StimOff_RSTN  StimOn_RSTN  \
0       24.893839    52.995316     17.426915      1.402081     4.568300   
1      129.267991   266.734242     71.873540      6.058359    21.048763   
2      114.126781   226.003336     41.669653      4.281059    15.290273   
3       58.517801   113.862102     32.059823      2.966395     5.927611   
4       28.949086    43.699708     20.393250      1.902084     2.732625   
..            ...          ...           ...           ...          ...   
121      0.000279     0.000233      0.000340      0.000213     0.000258   
122      0.000278     0.000200      0.000282      0.000278     0.000364   
123      0.000220     0.000340      0.000336      0.000411     0.000392   
124      1.023674     2.012760      0.030782      2.170008     2.625512   
125      2.042283     4.030933      0.062421      4.346489     5.268047   

     Rebound_RSTN  
0        3.452051  
1       11.505215  
2        3.433943  
3        2.708942  

In [35]:
fft_name = 'Sub035_3mfu_PS'

In [36]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','Power_Spectra','test/')
fft_file = os.path.join(project_path, 'data','power_spectra','test/')

plt.savefig(str(fft_fig)+str(fft_name), dpi = 150)

ps_df.to_csv(str(fft_file)+str(fft_name)+ '.csv')